#CUADERNO CPU

#Función para leer el set de datos

In [15]:
import numpy as np
import os
import gzip

def cargar_set(ruta, tipo='train'):

  ruta_categorias=os.path.join(ruta, '%s-labels-idx1-ubyte.gz' %tipo)
  ruta_imagenes = os.path.join(ruta, '%s-images-idx3-ubyte.gz' %tipo)
  with gzip.open(ruta_categorias, 'rb') as rut_cat:
      etiquetas = np.frombuffer(rut_cat.read(), dtype=np.uint8, offset=8)
  with gzip.open(ruta_imagenes, 'rb') as rut_imgs:
    imagenes = np.frombuffer(rut_imgs.read(), dtype=np.uint8, offset = 16).reshape(len(etiquetas), 784)
  return imagenes, etiquetas


#Acceso a Google Drive

In [16]:
from google.colab import drive
drive.mount('/content/gdrive')
ruta = '/content/gdrive/MyDrive/Diplomado/Entendiendo Google-Colab/2. Entendiendo Google Colab/fashion_mnist_data'

X_train, Y_train = cargar_set(ruta, tipo= 'train')
X_test, Y_test = cargar_set(ruta, tipo= 'test')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


#Reajustar el tamaño de los datos

In [20]:
#La arquitectura TPU recibe datos con tamaños multiplos de 128.

X_train = X_train[0:59904,:]
X_test = X_test[0:9984,:]

Y_train = Y_train[0:59904]
Y_test = Y_test[0:9984]

X_train = np.reshape(X_train, (59904,28,28,1))
X_test = np.reshape(X_test, (9984,28,28,1))

#Importar Tensorflow 2 (ya incluye keras)

In [23]:
%tensorflow_version 2.x
import tensorflow as tf

print('Versión de Tensorflow: ' + tf.__version__)

Versión de Tensorflow: 2.4.1


#Modelo

In [26]:
tf.random.set_seed(200)

modelo = tf.keras.models.Sequential()

modelo.add(tf.keras.layers.BatchNormalization(input_shape = X_train.shape[1:]))
modelo.add(tf.keras.layers.Conv2D(64,(5,5),padding='same',activation='elu'))
modelo.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)))
modelo.add(tf.keras.layers.Dropout(0.25)) #reduccion del overfitting del modelo

modelo.add(tf.keras.layers.BatchNormalization(input_shape = X_train.shape[1:]))
modelo.add(tf.keras.layers.Conv2D(128,(5,5),padding='same',activation='elu'))
modelo.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)))
modelo.add(tf.keras.layers.Dropout(0.25))

modelo.add(tf.keras.layers.BatchNormalization(input_shape = X_train.shape[1:]))
modelo.add(tf.keras.layers.Conv2D(256,(5,5),padding='same',activation='elu'))
modelo.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)))
modelo.add(tf.keras.layers.Dropout(0.25))

#Aplanar los datos
modelo.add(tf.keras.layers.Flatten())
modelo.add(tf.keras.layers.Dense(256)) #256 neuronas
modelo.add(tf.keras.layers.Activation('elu'))
modelo.add(tf.keras.layers.Dropout(0.5))
modelo.add(tf.keras.layers.Dense(10)) #Neuronas de salida
modelo.add(tf.keras.layers.Activation('softmax'))

#imprimir en pantalla información del modelo
modelo.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 28, 28, 1)         4         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 64)        1664      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 64)        256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 128)       204928    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 128)         0

# Entrenamiento de CPU




In [27]:
modelo.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [28]:
import timeit

def entrenamiento_cpu():
  with tf.device('/cpu:0'):
    modelo.fit(X_train,Y_train, validation_data=(X_test,Y_test),batch_size=128, epochs=2,verbose=1)

    return None

cpu_time =timeit.timeit('entrenamiento_cpu()',number=1, setup = 'from __main__ import entrenamiento_cpu')


Epoch 1/2
468/468 [==============================] - 685s 1s/step - loss: 1.0169 - accuracy: 0.7078 - val_loss: 0.4264 - val_accuracy: 0.8551
Epoch 2/2
468/468 [==============================] - 683s 1s/step - loss: 0.4105 - accuracy: 0.8562 - val_loss: 0.3127 - val_accuracy: 0.8922


In [29]:
print('Tiempo de entrenamiento '+str(cpu_time) + ' segundos')

Tiempo de entrenamiento 1368.6541898260002 segundos
